## 0. Download dataset

In [ ]:
# way 1
!wget https://motchallenge.net/data/MOT17.zip

# way 2: might not be available
# !gdown 1vOj9OpxeyozWzpPCtUY7fDVaBQwsPM9n

Downloading...
From (uriginal): https://drive.google.com/uc?id=1vOj9OpxeyozWzpPCtUY7fDVaBQwsPM9n
From (redirected): https://drive.google.com/uc?id=1vOj9OpxeyozWzpPCtUY7fDVaBQwsPM9n&confirm=t&uuid=f6c0598b-accf-473e-95b2-8be35b5c7932
To: /home/aivn12s1/thangdd/project_object_tracking/MOT17.zip
100%|██████████████████████████████████████| 5.86G/5.86G [09:50<00:00, 9.92MB/s]


In [1]:
!unzip -qq MOT17.zip

## 1. Import libraries

In [ ]:
!pip install ultralytics -q

In [1]:
import pandas as pd
import os
import yaml
import shutil
import configparser
import ultralytics
ultralytics.checks()

from tqdm import tqdm
from ultralytics import YOLO

Ultralytics YOLOv8.1.16 🚀 Python-3.8.18 torch-2.2.0+cu121 CPU (Intel Xeon E5-2686 v4 2.30GHz)
Setup complete ✅ (36 CPUs, 46.9 GB RAM, 34.1/337.6 GB disk)


## 2. Convert to YOLO format

In [2]:
def convert_to_yolo_format(bb, img_width, img_height):
    # format MOT17: (x_min, y_min, bb_w, bb_h)
    # format MOT17: (x_center, y_center, bb_w, bb_h) # normalized
    x_center = bb['bb_left'] + (bb['bb_width'] / 2)
    y_center = bb['bb_top'] + (bb['bb_height'] / 2)

    # Normalize the coordinates by the dimensions of the image
    x_center /= img_width
    y_center /= img_height
    bb_width_normalized = bb['bb_width'] / img_width
    bb_height_normalized = bb['bb_height'] / img_height

    # Clip the values to make sure they are between 0 and 1
    x_center = max(min(x_center, 1), 0)
    y_center = max(min(y_center, 1), 0)
    bb_width_normalized = max(min(bb_width_normalized, 1), 0)
    bb_height_normalized = max(min(bb_height_normalized, 1), 0)

    return (x_center, y_center, bb_width_normalized, bb_height_normalized)

In [3]:
def process_folder(folder_path):
    # Read image dimensions from seqinfo.ini
    config = configparser.ConfigParser()
    config.read(os.path.join(folder_path, 'seqinfo.ini'))
    img_width = int(config['Sequence']['imWidth'])
    img_height = int(config['Sequence']['imHeight'])

    # Load ground truth data
    gt_path = os.path.join(folder_path, 'det/det.txt')
    gt_data = pd.read_csv(
        gt_path,
        header=None,
        names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'class', 'visibility']
    )

    labels_folder = os.path.join(folder_path, 'labels')
    os.makedirs(labels_folder, exist_ok=True)

    for frame_number in gt_data['frame'].unique():
        frame_data = gt_data[gt_data['frame'] == frame_number]
        label_file = os.path.join(labels_folder, f'{frame_number:06d}.txt')

        with open(label_file, 'w') as file:
            for _, row in frame_data.iterrows():
                yolo_bb = convert_to_yolo_format(row, img_width, img_height)
                file.write(f'0 {yolo_bb[0]} {yolo_bb[1]} {yolo_bb[2]} {yolo_bb[3]}\n')

In [4]:
def process_all_folders(base_directory):
    # List all subdirectories in the base directory
    for folder_name in tqdm(os.listdir(base_directory)):
        folder_path = os.path.join(base_directory, folder_name)

        # Delete folder not contain 'FRCNN' in name
        if 'FRCNN' not in folder_name:
            os.system(f'rm -rf {folder_path}')
            continue

        if os.path.isdir(folder_path):
            process_folder(folder_path)

In [5]:
process_all_folders('MOT17/train')
process_all_folders('MOT17/test')

100%|██████████| 21/21 [00:12<00:00,  1.69it/s]


## 3. Move file

In [6]:
def rename_and_move_files(src_folder, dst_folder, folder_name, file_extension):
    for filename in os.listdir(src_folder):
        if filename.endswith(file_extension):
            # Include folder name in the new filename
            new_filename = f'{folder_name}_{filename}'
            shutil.move(os.path.join(src_folder, filename), os.path.join(dst_folder, new_filename))

In [7]:
def move_files_all_folders(base_directory):
    images_dir = os.path.join(base_directory, 'images')
    labels_dir = os.path.join(base_directory, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for folder_name in tqdm(os.listdir(base_directory)):
        if folder_name in ['images', 'labels']:  # Skip these folders
            continue

        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path):
            rename_and_move_files(os.path.join(folder_path, 'img1'), images_dir, folder_name, '.jpg')
            rename_and_move_files(os.path.join(folder_path, 'labels'), labels_dir, folder_name, '.txt')

In [8]:
move_files_all_folders('MOT17/train')
move_files_all_folders('MOT17/test')

100%|██████████| 9/9 [00:00<00:00, 29.06it/s]


In [9]:
def delete_subfolders(base_directory):
    for folder_name in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path) and folder_name not in ['images', 'labels']:
            shutil.rmtree(folder_path)
            print(f"Deleted folder: {folder_name}")

In [11]:
# Delete all subfolders except 'images' and 'labels'
delete_subfolders('MOT17/train')
delete_subfolders('MOT17/test')

## 4. Create yaml file

In [12]:
class_labels = [
    'objects'
]
dataset_root_dir = os.path.join(
    os.getcwd(),
    'MOT17'
)
yolo_yaml_path = os.path.join(
    dataset_root_dir,
    'mot17_data.yml'
)

data_yaml = {
    'path': dataset_root_dir,
    'train': 'train/images',
    'val': 'test/images',
    'nc': len(class_labels),
    'names': class_labels
}

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

## 5. Training

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

# Config
epochs = 30
batch_size = -1 # Auto scale based on GPU memory
img_size = 640
project_name = 'models/yolo'
name = 'yolov8s_mot17_det'

# Train the model
results = model.train(
    data=yolo_yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=project_name,
    name=name
)

## 6. Evaluation

In [ ]:
from ultralytics import YOLO

# Load the trained model
model_path = os.path.join(
    project_name, name, 'weights/best.pt'
)
model = YOLO(model_path)

metrics = model.val(
    project=project_name,
    name='detect/val'
)

Ultralytics YOLOv8.0.222 🚀 Python-3.11.5 torch-2.1.1 CUDA:0 (NVIDIA GeForce RTX 3060, 12036MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/aivn12s1/thangdd/project_object_tracking/MOT17/test/labels.cache... 5908 images, 11 backgrounds, 0 corrupt: 100%|██████████| 5919/5919 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 370/370 [00:33<00:00, 10.95it/s]


                   all       5919     110141      0.903      0.809      0.908      0.655
Speed: 0.1ms preprocess, 3.4ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to models/yolo/detect/val


## 7. Inference

In [ ]:
sample_path = 'MOT17/test/images/MOT17-01-FRCNN_000001.jpg'
results = model.predict(
    sample_path,
    project=project_name,
    name='detect/predict',
    save=True
)


image 1/1 /home/aivn12s1/thangdd/project_object_tracking/MOT17/test/images/MOT17-01-FRCNN_000001.jpg: 384x640 9 objectss, 4.8ms
Speed: 1.0ms preprocess, 4.8ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to models/yolo/detect/predict2
